In [1]:
# Necessary Imports
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("/home/lyrax/matplotlib-dracula/dracula.mplstyle")
import seaborn as sns
sns.set_palette("bright")

from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold

from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from tqdm import tqdm, trange
import warnings
warnings.filterwarnings("ignore")

In [18]:
# Load data
train = pd.read_csv("data/Train.csv", parse_dates=['created_at']); display(train.head())
test = pd.read_csv("data/Test.csv", parse_dates=['created_at']); display(test.head())

,ID,created_at,site,pm2_5,pm10,s2_pm2_5,s2_pm10,humidity,temp,lat,long,altitude,greenness,landform_90m,landform_270m,population,dist_major_road,ref_pm2_5
0,ID_0038MG0B,2020-04-23 17:00:00+03:00,USEmbassy,6.819048,7.313810,6.794048,7.838333,0.807417,22.383333,0.299255,32.592686,1199,4374,21,14,6834,130,25.0
1,ID_008ASVDD,2020-02-23 19:00:00+03:00,USEmbassy,57.456047,67.883488,55.643488,70.646977,0.712417,25.350000,0.299255,32.592686,1199,4374,21,14,6834,130,68.0
2,ID_009ACJQ9,2021-01-23 04:00:00+03:00,Nakawa,170.009773,191.153636,165.308636,191.471591,0.907833,20.616667,0.331740,32.609510,1191,5865,31,-11,4780,500,149.7
3,ID_00IGMAQ2,2019-12-04 09:00:00+03:00,USEmbassy,49.732821,61.512564,0.000000,0.000000,0.949667,21.216667,0.299255,32.592686,1199,4374,21,14,6834,130,54.0
4,ID_00P76VAQ,2019-10-01 01:00:00+03:00,USEmbassy,41.630455,51.044545,41.725000,51.141364,0.913833,18.908333,0.299255,32.592686,1199,4374,21,14,6834,130,39.0


,ID,created_at,site,pm2_5,pm10,s2_pm2_5,s2_pm10,humidity,temp,lat,long,altitude,greenness,landform_90m,landform_270m,population,dist_major_road
0,ID_00OZLF7X,2020-03-13 07:00:00+03:00,USEmbassy,31.900455,35.515455,31.672273,37.051818,0.927167,21.175000,0.299255,32.592686,1199,4374,21,14,6834,130
1,ID_00ZI0D98,2020-08-08 10:00:00+03:00,Makerere,53.581818,66.603636,50.586364,64.651818,0.811583,22.350000,0.333501,32.568561,1233,6340,21,28,8518,475
2,ID_017GTLAU,2020-08-25 09:00:00+03:00,Makerere,62.377500,71.647500,59.023333,69.766667,0.902000,20.766667,0.333501,32.568561,1233,6340,21,28,8518,475
3,ID_01IBM7T2,2020-06-15 16:00:00+03:00,USEmbassy,33.310294,36.958824,33.060882,38.674412,0.643417,25.483333,0.299255,32.592686,1199,4374,21,14,6834,130
4,ID_01II27D4,2021-01-13 00:00:00+03:00,Nakawa,64.782045,75.247500,64.638182,77.108864,0.939667,20.133333,0.331740,32.609510,1191,5865,31,-11,4780,500


In [19]:
# datasets have very few NaNs - impute with mean
train.humidity.fillna(train.humidity.mean(), inplace=True); train.temp.fillna(train.temp.mean(), inplace=True)
test.humidity.fillna(test.humidity.mean(), inplace=True); test.temp.fillna(test.temp.mean(), inplace=True)

## **Feature Engineering**

In [20]:
def feat_eng(df):
    le = LabelEncoder()
    kmeans = KMeans(n_clusters=3)
    
    # date features
    df['creation_year'] = pd.to_datetime(df.created_at).dt.year
    df['creation_month'] = pd.to_datetime(df.created_at).dt.month
    df['creation_day'] = pd.to_datetime(df.created_at).dt.day
    df['creation_hour'] = pd.to_datetime(df.created_at).dt.hour
    df['creation_weekday'] = pd.Series(pd.to_datetime(df.created_at).dt.weekday).apply(lambda x: 1 if x<5 else 0) # if weekday or weekeend
    df['days_todate'] = pd.to_datetime(pd.Timestamp.now().date()) - pd.to_datetime(df.created_at)
    df.days_todate = df.days_todate.apply(lambda x: int(str(x).split(' days')[0])) # remove string 'days'
    df['years_todate'] = df.days_todate.apply(lambda x: int(np.round(x/364)))
    
    # time difference(number of hours to next recording)
    timediff = df.sort_values('created_at').created_at.diff(); timediff.fillna(pd.Timedelta(seconds=0), inplace=True)
    timediff = timediff.apply(lambda val: str(val).split('days')[-1].split(':')[0])
    df['timediff'] = 0
    df.sort_values('created_at', inplace=True)
    df['timediff'] = timediff.astype(int); df.sort_index(inplace=True)
    
    # label encoded features
    df['site_le'] = le.fit_transform(df.site)
    df['creation_year_le'] = le.fit_transform(df.creation_year)
    
    
    # kmeans features
    kmeans.fit(df[['lat', 'long']])
    df['latlong_cluster'] = kmeans.labels_
    
    kmeans.fit(df[['landform_90m', 'landform_270m']])
    landform_labels = kmeans.labels_
    df['landforms_cluster'] = landform_labels
    
    kmeans.fit(pd.concat([pd.Series(landform_labels), pd.Series(df['greenness'].values)], axis=1))
    df['landforms_greenness_clusters'] = kmeans.labels_
    
    kmeans.fit(df[['altitude', 'greenness', 'population']])
    df['altgreenpop_cluster'] = kmeans.labels_
    
    kmeans.fit(df[['site_le', 'humidity', 'temp', 'lat', 'long', 'altitude', 'landform_90m',
                   'landform_270m', 'population', 'dist_major_road']])
    df['master_cluster'] = kmeans.labels_
    
    # dewpoint
    df['dewpoint'] = 0
    for i, _ in tqdm(enumerate(df.temp)):
        temp = df.loc[i, 'temp']
        humidity = df.loc[i, 'humidity']
        df.loc[i, 'dewpoint'] = temp - ((1 - humidity)/5)
        
    
    # temperature and humidity categories
    # temp
    conditions = [
        (df['temp'] <= 20),
        (df['temp'] > 20) & (df['temp'] <= 25),
        (df['temp'] > 25)]
    
    values = ['2', '3', '1']
    df['temp_cat'] = np.select(conditions, values).astype(int)
    
    # temp
    conditions = [
    (df['humidity'] <= 0.5),
    (df['humidity'] > 0.5) & (df['humidity'] <= 0.8),
    (df['humidity'] > 0.8)]

    values = ['1', '2', '3']
    df['humidity_cat'] = np.select(conditions, values).astype(int)
    
    # combination features
    df['humixtemp'] = df.humidity * df.temp
    df['humixtemp2'] = (df.humidity*100) / df.temp
    df['altxhumi'] = df.altitude * df.humidity
    df['altxtemp'] = df.altitude * df.temp
    
    # differences between pm2_5:s2_pm2_5, pm10:s2_pm10 and between pm2_5's:pm10's
    df['pm2_5_diff'] = np.abs(df.pm2_5 - df.s2_pm2_5)
    df['pm10_diff'] = np.abs(df.pm10 - df.s2_pm10)
    df['pm10xpm25'] = np.abs(df.pm10 - df.pm2_5)
    df['s2_pm10xpm25'] = np.abs(df.s2_pm10 - df.s2_pm2_5)
    
    df['pm_total_diffs'] = df.pm2_5_diff + df.pm10_diff + df.pm10xpm25 + df.s2_pm10xpm25
    df['pm_diffs_mean'] = (df.pm2_5_diff + df.pm10_diff + df.pm10xpm25 + df.s2_pm10xpm25)/4
    
    # humixtemp categories
    conditions = [
    (df['humixtemp'] <= 8),
    (df['humixtemp'] > 13) & (df['humixtemp'] <= 18),
    (df['humixtemp'] > 18)]

    values = ['2', '1', '3']
    df['humixtemp_cat'] = np.select(conditions, values).astype(int)
    
    # altxhumi categories
    conditions = [
    (df['altxhumi'] <= 600),
    (df['altxhumi'] > 600) & (df['altxhumi'] <= 900),
    (df['altxhumi'] > 900)]
    values = ['3', '1', '2']
    df['altxhumi_cat'] = np.select(conditions, values).astype(int)
    
    # altxtemp categories
    conditions = [
    (df['altxtemp'] <= 25000),
    (df['altxtemp'] > 25000) & (df['altxtemp'] <= 31000),
    (df['altxtemp'] > 31000)]
    values = ['3', '1', '2']
    df['altxtemp_cat'] = np.select(conditions, values).astype(int)
    
    # pca feature
    pca = PCA(n_components=1, random_state=101)
    vals = pca.fit_transform(df[['pm2_5', 'pm10', 's2_pm2_5', 's2_pm10',
       'humidity', 'temp', 'lat', 'long', 'altitude', 'greenness',
       'landform_90m', 'landform_270m', 'population', 'dist_major_road',
       'creation_month', 'creation_day',
       'creation_hour', 'creation_weekday', 'days_todate', 'years_todate',
       'timediff', 'site_le', 'creation_year_le', 'latlong_cluster',
       'landforms_cluster', 'landforms_greenness_clusters',
       'altgreenpop_cluster', 'master_cluster', 'dewpoint', 'temp_cat',
       'humidity_cat', 'humixtemp', 'humixtemp2', 'altxhumi', 'altxtemp',
       'pm2_5_diff', 'pm10_diff', 'pm10xpm25', 's2_pm10xpm25',
       'pm_total_diffs', 'pm_diffs_mean', 'humixtemp_cat', 'altxhumi_cat',
       'altxtemp_cat']])
    df['component'] = pd.Series(vals.reshape(vals.shape[0],))
    
    # component categories
    # altxtemp categories
    conditions = [
    (df['component'] <= -1000),
    (df['component'] > -1000) & (df['component'] <= 5000),
    (df['component'] > 5000)]
    values = ['2', '1', '3']
    df['component_cat'] = np.select(conditions, values).astype(int)
    
    

In [21]:
# join data, apply functions, separate data
data = pd.concat([train, test]).reset_index(drop=True)

# localize timezone to one
data.created_at = data['created_at'].dt.tz_localize(None)

feat_eng(data)

train = data[data.ref_pm2_5.notnull()].reset_index(drop=True)
test = data[data.ref_pm2_5.isna()].reset_index(drop=True); test.drop('ref_pm2_5', axis=1, inplace=True)

display(train.head())
display(test.head())

13665it [00:05, 2600.48it/s]


,ID,created_at,site,pm2_5,pm10,s2_pm2_5,s2_pm10,humidity,temp,lat,long,altitude,greenness,landform_90m,landform_270m,population,dist_major_road,ref_pm2_5,creation_year,creation_month,creation_day,creation_hour,creation_weekday,days_todate,years_todate,timediff,site_le,creation_year_le,latlong_cluster,landforms_cluster,landforms_greenness_clusters,altgreenpop_cluster,master_cluster,dewpoint,temp_cat,humidity_cat,humixtemp,humixtemp2,altxhumi,altxtemp,pm2_5_diff,pm10_diff,pm10xpm25,s2_pm10xpm25,pm_total_diffs,pm_diffs_mean,humixtemp_cat,altxhumi_cat,altxtemp_cat,component,component_cat
0,ID_0038MG0B,2020-04-23 17:00:00,USEmbassy,6.819048,7.313810,6.794048,7.838333,0.807417,22.383333,0.299255,32.592686,1199,4374,21,14,6834,130,25.0,2020,4,23,17,1,371,1,1,2,1,0,2,1,0,0,22.344817,3,3,18.072676,3.607223,968.092584,26837.616663,0.025000,0.524524,0.494762,1.044286,2.088571,0.522143,3,2,1,306.800537,1
1,ID_008ASVDD,2020-02-23 19:00:00,USEmbassy,57.456047,67.883488,55.643488,70.646977,0.712417,25.350000,0.299255,32.592686,1199,4374,21,14,6834,130,68.0,2020,2,23,19,0,431,1,1,2,1,0,2,1,0,0,25.292483,1,2,18.059763,2.810322,854.187584,30394.650000,1.812558,2.763488,10.427442,15.003488,30.006977,7.501744,3,1,1,3861.874705,1
2,ID_009ACJQ9,2021-01-23 04:00:00,Nakawa,170.009773,191.153636,165.308636,191.471591,0.907833,20.616667,0.331740,32.609510,1191,5865,31,-11,4780,500,149.7,2021,1,23,4,0,96,0,1,1,2,2,1,2,2,2,20.598233,3,3,18.716497,4.403395,1081.229500,24554.450004,4.701136,0.317955,21.143864,26.162955,52.325909,13.081477,3,2,3,-2055.336813,2
3,ID_00IGMAQ2,2019-12-04 09:00:00,USEmbassy,49.732821,61.512564,0.000000,0.000000,0.949667,21.216667,0.299255,32.592686,1199,4374,21,14,6834,130,54.0,2019,12,4,9,1,512,1,1,2,0,0,2,1,0,0,21.206600,3,3,20.148761,4.476041,1138.650334,25438.783337,49.732821,61.512564,11.779744,0.000000,123.025128,30.756282,3,2,1,-1097.933751,2
4,ID_00P76VAQ,2019-10-01 01:00:00,USEmbassy,41.630455,51.044545,41.725000,51.141364,0.913833,18.908333,0.299255,32.592686,1199,4374,21,14,6834,130,39.0,2019,10,1,1,1,576,2,1,2,0,0,2,1,0,0,18.891100,2,3,17.279065,4.832966,1095.686166,22671.091663,0.094545,0.096818,9.414091,9.416364,19.021818,4.755455,1,2,3,-3859.598195,2


,ID,created_at,site,pm2_5,pm10,s2_pm2_5,s2_pm10,humidity,temp,lat,long,altitude,greenness,landform_90m,landform_270m,population,dist_major_road,creation_year,creation_month,creation_day,creation_hour,creation_weekday,days_todate,years_todate,timediff,site_le,creation_year_le,latlong_cluster,landforms_cluster,landforms_greenness_clusters,altgreenpop_cluster,master_cluster,dewpoint,temp_cat,humidity_cat,humixtemp,humixtemp2,altxhumi,altxtemp,pm2_5_diff,pm10_diff,pm10xpm25,s2_pm10xpm25,pm_total_diffs,pm_diffs_mean,humixtemp_cat,altxhumi_cat,altxtemp_cat,component,component_cat
0,ID_00OZLF7X,2020-03-13 07:00:00,USEmbassy,31.900455,35.515455,31.672273,37.051818,0.927167,21.175000,0.299255,32.592686,1199,4374,21,14,6834,130,2020,3,13,7,1,412,1,1,2,1,0,2,1,0,0,21.160433,3,3,19.632754,4.378591,1111.672834,25388.825000,0.228182,1.536364,3.615000,5.379545,10.759091,2.689773,3,2,1,-1146.087460,2
1,ID_00ZI0D98,2020-08-08 10:00:00,Makerere,53.581818,66.603636,50.586364,64.651818,0.811583,22.350000,0.333501,32.568561,1233,6340,21,28,8518,475,2020,8,8,10,0,264,1,1,0,1,1,0,0,1,1,22.312317,3,3,18.138887,3.631245,1000.682250,27557.550000,2.995455,1.951818,13.021818,14.065455,32.034545,8.008636,3,2,1,1090.642107,1
2,ID_017GTLAU,2020-08-25 09:00:00,Makerere,62.377500,71.647500,59.023333,69.766667,0.902000,20.766667,0.333501,32.568561,1233,6340,21,28,8518,475,2020,8,25,9,1,247,1,1,0,1,1,0,0,1,1,20.747067,3,3,18.731533,4.343499,1112.166000,25605.300004,3.354167,1.880833,9.270000,10.743333,25.248333,6.312083,3,2,1,-863.059255,1
3,ID_01IBM7T2,2020-06-15 16:00:00,USEmbassy,33.310294,36.958824,33.060882,38.674412,0.643417,25.483333,0.299255,32.592686,1199,4374,21,14,6834,130,2020,6,15,16,1,318,1,1,2,1,0,2,1,0,0,25.412017,1,2,16.396401,2.524853,771.456584,30554.516663,0.249412,1.715588,3.648529,5.613529,11.227059,2.806765,1,1,1,4025.722348,1
4,ID_01II27D4,2021-01-13 00:00:00,Nakawa,64.782045,75.247500,64.638182,77.108864,0.939667,20.133333,0.331740,32.609510,1191,5865,31,-11,4780,500,2021,1,13,0,1,107,0,1,1,2,2,1,2,2,2,20.121267,3,3,18.918622,4.667219,1119.143000,23978.799996,0.143864,1.861364,10.465455,12.470682,24.941364,6.235341,3,2,3,-2630.416201,2


### **PCA Feature**

In [50]:
pca = PCA(n_components=1, random_state=101)
vals = pca.fit_transform(train[feat_cols])
pd.Series(vals.reshape(vals.shape[0],))

In [22]:
# Feature columns

feat_cols = ['pm2_5', 'pm10', 's2_pm2_5', 's2_pm10',
       'humidity', 'temp', 'lat', 'long', 'altitude', 'greenness',
       'landform_90m', 'landform_270m', 'population', 'dist_major_road',
       'creation_month', 'creation_day',
       'creation_hour', 'creation_weekday', 'days_todate', 'years_todate',
       'timediff', 'site_le', 'creation_year_le', 'latlong_cluster',
       'landforms_cluster', 'landforms_greenness_clusters',
       'altgreenpop_cluster', 'master_cluster', 'dewpoint', 'temp_cat',
       'humidity_cat', 'humixtemp', 'humixtemp2', 'altxhumi', 'altxtemp',
       'pm2_5_diff', 'pm10_diff', 'pm10xpm25', 's2_pm10xpm25',
       'pm_total_diffs', 'pm_diffs_mean', 'humixtemp_cat', 'altxhumi_cat',
       'altxtemp_cat', 'component', 'component_cat']

In [8]:
#test[feat_cols]

## **Modeling**

In [23]:
X = train[feat_cols]
y = train.ref_pm2_5

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [24]:
model = CatBoostRegressor(random_seed=101, silent=True, eval_metric='RMSE', learning_rate=0.05, n_estimators=1500)
model.fit(X_train, y_train)
pred = model.predict(X_test)
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, pred))}")

RMSE: 11.11792564018084


In [25]:
model = XGBRegressor(random_state=101, learning_rate=0.07, n_estimators=500)
model.fit(X_train, y_train)
pred = model.predict(X_test)
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, pred))}")

RMSE: 12.325610772806211


In [26]:
model = LGBMRegressor(objective='rmse', random_state=101, n_estimators=200)
model.fit(X_train, y_train)
pred = model.predict(X_test)
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, pred))}")

RMSE: 11.661236486551207


## **Cross Validation**

In [27]:
error=[]
pred_test = np.zeros(len(test))

NFOLDS = 10
fold = KFold(n_splits=NFOLDS)

for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    cat  = CatBoostRegressor(verbose=False, random_seed=101, use_best_model=True, loss_function='RMSE', n_estimators=1500, learning_rate=0.05)
    cat.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)])
    preds=cat.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    error.append(np.sqrt(mean_squared_error(y_test,preds)))
    p2 = cat.predict(test[feat_cols])
    pred_test+=p2
np.mean(error)

err:  10.994331075387418
err:  11.445971445554752
err:  15.041268262783161
err:  14.157144305902806
err:  9.746221275158016
err:  11.929871881625383
err:  8.816524368588201
err:  8.870216473662532
err:  10.439905453672504
err:  15.622165887056685


11.706362042939146

In [28]:
# submission file
sub = pd.DataFrame({
    'ID': test.ID,
    'ref_pm2_5': pred_test/NFOLDS
})
sub.to_csv("submissions/sub24.csv", index=False)
sub.head()

,ID,ref_pm2_5
0,ID_00OZLF7X,40.982415
1,ID_00ZI0D98,40.386853
2,ID_017GTLAU,46.141249
3,ID_01IBM7T2,39.008233
4,ID_01II27D4,57.988767


In [ ]:
# LGBM
error=[]
pred_test = np.zeros(len(test))

NFOLDS = 10
fold = KFold(n_splits=NFOLDS)

for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    cat  = LGBMRegressor(objective='rmse', random_state=101, n_estimators=200)
    cat.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)])
    preds=cat.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    error.append(np.sqrt(mean_squared_error(y_test,preds)))
    p2 = cat.predict(test[feat_cols])
    pred_test+=p2
np.mean(error)

# submission file
sub = pd.DataFrame({
    'ID': test.ID,
    'ref_pm2_5': pred_test/NFOLDS
})
sub.to_csv("submissions/sub25.csv", index=False)
sub.head()

In [30]:
# Blends
sub1 = pd.read_csv("submissions/sub24.csv")
sub2 = pd.read_csv("submissions/sub25.csv")

blend = (sub1.ref_pm2_5 + sub2.ref_pm2_5)/2

In [31]:
sub = pd.DataFrame({
    'ID': test.ID,
    'ref_pm2_5': blend
})
sub.to_csv("submissions/sub26.csv", index=False)
sub.head()

,ID,ref_pm2_5
0,ID_00OZLF7X,40.106055
1,ID_00ZI0D98,40.644360
2,ID_017GTLAU,46.128550
3,ID_01IBM7T2,39.478064
4,ID_01II27D4,57.291212


In [32]:
# SCORES
# catboost : 9.78370497274929
# lgbm : 9.6688751131256

# catboost X lgbm : 9.59677474736922

# reduced 'component_cat' to 3 categories rather than 6
# catboost : 9.73782951918674
# lgbm : 9.69441912860714
# catboost X lgbm : 9.58056882913722